# Business Analytics und Künstliche Intelligenz
Wintersemester 2023/2024

Prof. Dr. Jürgen Bock

## Übungen zu den Grundlagen künstlicher Neuronaler Netze in PyTorch

Dieses Notebook bietet Übungsaufgaben zum grundlegenden Umgang mit künstlichen Neuronalen Netzen in *PyTorch*. Die einzelnen Aufgaben sind in Markdown-Zellen beschrieben. Fügen Sie Ihre Lösung in die jeweils nachfolgende Code-Zelle ein und fügen Sie bei Bedarf gerne weitere Code-Zellen hinzu.

### Lernziele
* Sie sind in der Lage einfache künstliche Neuronen zu implementieren und die Einflüsse der einzelnen Komponenten zu untersuchen.
* Sie sind in der Lage strukturierte Datensätze zur Verwendung in *PyTorch* vorzubereiten und einfache mehrschichtige künstliche neuronale Netze zu strukturieren.
* Sie sind in der Lage eigene neuronale Netzstrukturen in einen generischen Lernalgorithmus einzuordnen und mit verschiedenen Datensätzen zu trainieren.
* Sie sind in der Lage den Einfluss verschiedener Hyperparameter auf den Lernprozess und Qualität des gelernten Modells bei künstlichen neuronalen Netzen zu untersuchen und zu diskutieren.

### Einfache Neuronen

Betrachten Sie die Bool'sche Funktion *NAND*.

Die Wahrheitstafel für NAND ist wie folgt:

| NAND | 0 | 1 |
|-----|---|---| 
| **0**   | 1 | 1 |
| **1**   | 1 | 0 |

Warum ist diese Funktion durch ein einzelnes Neuron berechenbar?

*Antwort:* Sie ist linear separierbar, d.h. die Samples der einzelnen Klassen sind durch eine lineare Funktion trennbar. (Hier eine Gerade.)

Überlegen Sie sich wie dieses eine Neuron konfiguriert sein müsste. (Eingänge, Gewichte)

*Antwort:* Die beiden Eingänge werden negativ gewichtet, der Bias positiv. Dabei ist darauf zu achten, dass das Einfluss des Bias die Eingangssumme gerade dann größer als 0 werden lässt, wenn höchstens einer der beiden Eingänge 1 ist, also das Eingangsgewicht zur Geltung kommt. 

Implementieren Sie die Berechnung in Python und testen Sie Ihre Lösung.

In [ ]:
def threshold(x):
    if x < 0:
        return 0
    else:
        return 1

In [ ]:
x1 = 1
x2 = 1

w03 = 1.5
w13 = -1
w23 = -1

y = threshold(w03 + x1*w13 + x2*w23)
print('{} NAND {} -> {}'.format(x1, x2, y))

### Mehrschichtige neuronale Netze

#### Synthetische Daten

Betrachten Sie den folgenden (synthetischen) Datensatz mit zwei Merkmalen und zwei Klassen:

In [ ]:
from sklearn import datasets

data = datasets.make_circles(
    n_samples = 10000,
    noise = 0.1,
    factor = 0.5 )

Machen Sie sich mit dem Datensatz vertraut indem Sie einen Scatter-Plot erstellen.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
X, t = data
plt.scatter(X[:,0], X[:,1], c=t, s=1)
plt.show()

Definieren Sie ein mehrschichtiges neuronales Netz mittels *PyTorch*, und implementieren Sie eine Trainingsroutine.

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
dataset = TensorDataset(torch.from_numpy(X), torch.from_numpy(t))
data_loader = DataLoader(dataset=dataset, batch_size=5, shuffle=True)

In [ ]:
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [ ]:
class MLP( nn.Module ):
    def __init__( self ):
        super( MLP, self ).__init__()
        self.fc1 = nn.Linear( 2, 3 )
        self.fc3 = nn.Linear( 3, 1 )
        
    def forward( self, x ):
        x = torch.sigmoid( self.fc1( x ) )
        x = torch.sigmoid( self.fc3( x ) )
        return x

In [ ]:
model = MLP()

In [ ]:
num_epochs = 10

In [ ]:
loss_fn = nn.BCELoss()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
from IPython import display
from statistics import mean
loss_history = []
loss_ep = []
plt.figure(figsize = (12,8));

In [ ]:
for epoch in range(num_epochs):
    for batch in data_loader :
        optimizer.zero_grad()
        input, target = batch
        output = model(input.float())
        loss = loss_fn(output, torch.unsqueeze(target.float(), 1))
        loss.backward()
        optimizer.step()
        loss_ep.append(loss.item())
    
    ## Zu Visualisierungszwecken:
    loss_history.append(mean(loss_ep))
    loss_ep = []
    display.clear_output(wait=True)
    plt.plot(loss_history)
    #dataview.plot_decision_boundary2d(model, X, t, showData=False)
    display.display(plt.gcf())
    display.display(print("Epoch {:2}, loss: {}".format(epoch, loss_history[-1])))

Stellen Sie sicher, dass das Modul ``dataview`` im aktuellen Verzeichnis liegt (oder in ``sys.path``). Verwenden Sie die Funktion ``dataview.plot_decision_boundary2d(model, X, y)`` um die *decision boundary* darzustellen.

In [ ]:
import dataview

In [ ]:
dataview.plot_decision_boundary2d(model, X, t, showData=False)

Experimentieren Sie mit den sogenannten *Hyperparametern*: Verändern Sie Anzahl und Breite der Layer, verändern Sie Anzahl der Epochen und die *batch size*. Konsultieren Sie die *PyTorch* API Dokumentation und experimentieren Sie mit verschiedenen Aktivierungsfunktionen und Optimierern.

**Beachten Sie:** Nach Veränderung des Modells muss die Objektinstanz des Modells neu instanziiert werden (z.B. ``model = Net()``). Außerdem muss der Optimierer und etwaige Hilfsvariablen, wie z.B. eine ``loss_history`` o.ä., neu initialisiert werden.

Welche Erkenntnisse haben Sie erlangt?

*Antwort:*

- Es genügt ein Netzwerk mit einer kleinen *hidden layer* um dieses Klassifizierungsproblem zu lösen. 3 Neuronen in der *hidden layer* funktionieren. Je mehr Neuronen in der *hidden layer* umso genauer wird der Kreis approximiert.
- Die *sigmoid* Funktion als Aktivierungsfunktion funktioniert.
- Die Robustheit und Geschwindigkeit der Konvergenz hängt vom Optimierer ab. *Adam* funktioniert deutlich besser als *SGD*.
- Die Learning Rate ist für beide Optimierungsalgorithmen unterschiedlich effektiv. Dieses Klassifizierungsproblem ist relativ robust gegenüber größeren Learaning Rates. Offensichtlich gibt es keine großen Gefahren durch lokale Minima im Gewichtsraum.
- Eine kleine *batch size* (im Extremfall 1) führt zu einer deutlichen Verlangsamung der Iteration über die Epochen. Dafür ist bereits nach einer Epoche eine sehr gute Näherung gefunden. Eine große *batch size* führt zu einer schnellen Iteration über die Epochen, die Konvergenz verläuft langsamer. Damit wäre das Modell robuster gegenüber Overfitting ("Auswendiglernen des Trainingsdatensatzes"). Dieser Effekt ist hier aber nicht zu beobachten.

#### Reale Daten

Verwenden Sie ``scikit-learn`` um den *Breast Cancer Wisconsin* Datensatz zu laden. Siehe: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html

In [ ]:
from sklearn.datasets import load_breast_cancer

In [ ]:
data = load_breast_cancer()

In [ ]:
print('Anzahl features:', len(data.feature_names))
print(data.feature_names)
print('Anzahl Klassen:', len(data.target_names))
print(data.target_names)
print('Anzahl Samples:', len(data.data))

Machen Sie sich mit der Funktion ``train_test_split`` aus dem Modul ``sklearn.model_selection`` vertraut. Verwenden Sie diese Funktion um den Datensatz in eine Trainings- und eine Testmenge aufzuteilen.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, t_train, t_test = train_test_split(data.data, data.target)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

Definieren Sie ein neuronales Netz und verwenden Sie den Trainingsdatensatz um es zu trainieren. Beachten Sie bei der neuronalen Netzstruktur die Größe des Eingabe- und Ausgabevektors.

In [ ]:
from torch.utils.data import TensorDataset
import torch
from torch.utils.data import DataLoader

In [ ]:
dataset_train = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(t_train))

In [ ]:
data_loader = DataLoader(dataset=dataset_train, batch_size=10, shuffle=True)

In [ ]:
from torch import nn

In [ ]:
loss_fn = nn.BCELoss()

In [ ]:
from torch.nn import functional as F

class MLP(nn.Module):
    def __init__( self ):
        super( MLP, self ).__init__()
        self.fc1 = nn.Linear( 30, 50 )
        self.fc2 = nn.Linear( 50, 20 )
        self.fc3 = nn.Linear( 20, 5 )
        self.fc4 = nn.Linear( 5, 1 )
        
    def forward( self, x ):
        x = F.relu( self.fc1( x ) )
        x = F.relu( self.fc2( x ) )
        x = F.relu( self.fc3( x ) )
        x = torch.sigmoid( self.fc4( x ) )
        return x

In [ ]:
model = MLP()

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline

loss_history = []
loss_ep = []
plt.figure(figsize = (12,8));

In [ ]:
num_epochs = 100

In [ ]:
for epoch in range(num_epochs):
    for batch in data_loader :
        optimizer.zero_grad()
        input, target = batch
        output = model(input.float())
        loss = loss_fn(output, torch.unsqueeze(target.float(), 1))
        loss.backward()
        optimizer.step()
        loss_ep.append(loss.item())
    
    ## Zu Visualisierungszwecken:
    loss_history.append(mean(loss_ep))
    plt.plot(loss_history)
    display.clear_output(wait=True)
    display.display(plt.gcf())
    display.display(print("Epoch {:2}, loss: {}".format(epoch, loss_history[-1])))
    loss_ep = []

Verwenden Sie ``scikit-learn`` um einen *classification report* zu erstellen, anhand dessen Sie ihr Modell bewerten können.

Berechnen Sie dazu zuerst den Ausgangsvektor ihres neuronalen Netzes für die Eingabevektoren des Testdatensatzes. 

**Hinweis:** ``torch.from_numpy`` erstellt einen Tensor aus einem *NumPy*-Array, in dem der Testdatensatz vorliegt. Zudem müssen Sie den Eingabevektor in einen ``FloatTensor`` konvertieren.

Für den *classification report* müssen Sie außerdem die Fließkommazahlen des Ausgangsvektors (resultierend aus der ``sigmoid`` Aktivierungsfunktion) in einen ganzzahlige Werte umwandeln.

In [ ]:
y_test = model(torch.from_numpy(X_test).float())

In [ ]:
import sklearn.metrics as metrics

In [ ]:
print(metrics.classification_report(t_test, torch.round(y_test).int()))

In [ ]:
print(metrics.confusion_matrix(t_test, torch.round(y_test).int()))